#### 1. 필요한 파일 임포트

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
# os: 파일, 폴더 처리에 관련된 라이브러리
import os

### 2. 데이터 수집 및 전처리

In [10]:
# 곱창과 떡갈비 폴더의 train, test 데이터 불러오기
train_gob_dir = 'data/gobchang_train/'
train_gal_dir = 'data/galbi_train/'
test_gob_dir = 'data/gobchang_test/'
test_gal_dir = 'data/galbi_test/'

In [11]:
# os.listdir() : 해당 파일 경로에 있는 파일명들을 리스트에 순서대로 저장
train_gob_fnames = os.listdir(train_gob_dir)
train_gal_fnames = os.listdir(train_gal_dir)
test_gob_fnames = os.listdir(test_gob_dir)
test_gal_fnames = os.listdir(test_gal_dir)

In [12]:
# os.path.join : 파일 경로와 파일명을 결합
test_path = os.path.join(train_gob_dir, train_gob_fnames[0])
test_path

'data/gobchang_train/0.jpg'

In [13]:
# 사진을 로딩하는 함수를 만들어보자!
# 사진 크기는 전부 224 x 224로 고정시키자
# 신경망에 넣기위해서는 데이터의 크기를 같게 만들어줘야 함

def load_images(folder_path, file_names, img_size_shape=(224,224)) :
    images = []
    
    for i in file_names :
        # 폴더 경로 + 파일명 합치기
        path = os.path.join(folder_path, i)
        # 파일 오픈 및 크기조정(resize : 파일의 형태(사이즈)를 변형시켜줌)
        img = Image.open(path).resize(img_size_shape)
        # numpy 배열로 변경 후 리스트에 추가
        images.append(np.array(img))
        
    # 리스트도 numpy 배열로 변환하여 리턴
    return np.array(images)

In [14]:
X_train_gob = load_images (train_gob_dir, train_gob_fnames)
X_train_gal = load_images (train_gal_dir, train_gal_fnames)
X_test_gob = load_images (test_gob_dir, test_gob_fnames)
X_test_gal = load_images (test_gal_dir, test_gal_fnames)

In [17]:
print(X_train_gob.shape)
print(X_train_gal.shape)
print(X_test_gob.shape)
print(X_test_gal.shape)

(200, 224, 224, 3)
(200, 224, 224, 3)
(150, 224, 224, 3)
(150, 224, 224, 3)


In [18]:
# 데이터 병합 (곱창, 갈비 이미지 배열 데이터를 순서대로 합쳐줌)
X_train = np.concatenate((X_train_gob, X_train_gal))
X_test = np.concatenate((X_test_gob,X_test_gal))


X_train.shape, X_test.shape

((400, 224, 224, 3), (300, 224, 224, 3))

In [19]:
# 정답 데이터 만들기 (곱창 : 0, 갈비 : 1)
y_train = np.array([0]*200 + [1]*200)
y_test = np.array([0]*150 + [1]*150)

In [20]:
# 0~255 사이의 픽셀 값을 0~1사이로 줄여줄 수 있음
# 수치값이 작아지기 때문에 연산속도를 향상시킬 수 있음
# 데이터의 픽셀 수치값들을 실수로 변경 시킨 후에 나눠줌
X_train = X_train.astype("float32") /255
X_test = X_test.astype ("float32") /255

In [21]:
# NPZ 파일로 저장
np.savez_compressed ('data/gobs_and_gals.npz', # 폴더 경로와 파일명을 설정
                    X_train = X_train,           # 학습용 문제 데이터 설정
                    X_test = X_test,             # 평가용 문제 데이터 설정
                    y_train = y_train,           # 학습용 정답 데이터 설정
                    y_test = y_test              # 평가용 정답 데이터 설정
                    )

### 3.이미지 파일 불러오기

In [23]:
data = np.load('data/gobs_and_gals.npz')
data

In [24]:
len(data)

4

In [25]:
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

In [26]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(400, 224, 224, 3)
(300, 224, 224, 3)
(400,)
(300,)


### 4. CNN 신경망 모델링

In [28]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import VGG16

In [1]:
# 신경망 구조 설계
gob_gal_model = Sequential()

# 1. 특성추출부(Conv - 특징이 되는 부분의 정보를 부각시킴)
# Conv2D : 2D 이미지 데이터에 대해 사용
# MaxPooling2D : 2D 이미지 데이터에 대해 필요없는 부분을 삭제
gob_gal_model.add(Conv2D(input_shape=(224,224,3),
                        # 필터의 개수 -> 추출하는 특징의 개수를 설정
                        filters=64,
                        # 필터(커널)의 크기를 설정
                        kernel_size = (5,5),
                        # same : 원본 데이터의 크기에 맞게 알아서 패딩을 적용(valid : 패딩 적용 X)
                        padding = 'same',
                        # 필터로 합성곱 연산시 건너뛰는 크기를 지정(행,열), 일반적으로 필터 사이즈보다 작게 설정
                        strides = (3,3),
                        activation = 'relu'                         
                        ))

gob_gal_model.add(Conv2D(filters=64,
                        kernel_size = (3,3),
                        padding = 'same',
                        activation = 'relu'
                        ))

gob_gal_model.add(Dropout(0.25))

# 2. 특성추출부(Pooling - 불필요한 부분을 삭제)
# Pool_size : 풀링층 필터의 크기
gob_gal_model.add(MaxPool2D(pool_size=2,
                            strides = (2,2)
                            ))

# MLP 층에 넣어주기
gob_gal_model.add(Flatten()) # input_shape 사용하지 않아도 됨, 자동으로 전체 길이만큼 펴줌
gob_gal_model.add(Dense(300, activation='relu'))
gob_gal_model.add(Dense(150, activation='relu'))
gob_gal_model.add(Dense(50, activation='relu'))
gob_gal_model.add(Dense(1, activation='sigmoid'))

gob_gal_model.summary()

 # CNN층에서의 파라미터수는 필터크기 * 필터수 * 채널수 + 필터수

NameError: name 'Sequential' is not defined

In [29]:

vgg16 = VGG16(include_top=False,
            weights = 'imagenet',
            input_shape = (224,224,3)
            )

# include_top = False : 불러온 모델의 MLP층을 사용하지 않음 (특성추출방식)
# -> 기존 이미지넷 대회에서는 1000가지 이미지를 분류했으나 우리는 개, 고양이로만 분류할 것이기 때문에 MLP층을 다르게 설정해줘야 함
# weights = 'imagenet' : 이미지넷에서 학습된 가중치(w)를 그대로 가져옴

58900480/58889256 [==============================] - 1s 0us/step


In [30]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [31]:
# 신경망 설계
transfer_model = Sequential()

transfer_model.add(vgg16)

# 미세조정방식 적용
# VGG16 모델에서 마지막 conv층을 학습이 가능하도록 설정해보자!
# 마지막 층을 제외한 나머지는 모두 동결시켜서 기존 학습된 w,b값을 보존하자!
# --------------------------------------------
for layer in vgg16.layers:
    # block5_conv3 층만 학습이 가능하도록 설정
    if layer .name == 'block5_conv3':
        layer.trainable = True
    # 나머지 층들은 학습이 불가능하도록 동결
    else:
        layer.trainable = False
# --------------------------------------------

transfer_model.add(Flatten())
transfer_model.add(Dense(300, activation='relu'))
transfer_model.add(Dense(150, activation='relu'))
transfer_model.add(Dense(50, activation='relu'))
transfer_model.add(Dense(1, activation='sigmoid'))

transfer_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 300)               7526700   
_________________________________________________________________
dense_1 (Dense)              (None, 150)               45150     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                7550      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 22,294,139
Trainable params: 9,939,259
Non-trainable params: 12,354,880
____________________________________

In [32]:
# 학습/평가 방법 설정
transfer_model.compile(loss = 'binary_crossentropy',
                    optimizer=Adam(lr=0.0005),
                    metrics = ['acc']
                    )

C:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [33]:
h = transfer_model.fit(X_train, y_train,
                    validation_split = 0.2,
                    epochs = 30,
                    batch_size = 128
                    )

Epoch 1/30
3/3 [==============================] - 67s 20s/step - loss: 1.2372 - acc: 0.5719 - val_loss: 0.2562 - val_acc: 1.0000
Epoch 2/30
1/3 [=========>....................] - ETA: 33s - loss: 0.6282 - acc: 0.4453

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(h2.history['acc'],
        label = 'acc'
        )

plt.plot(h2.history['val_acc'],
        label = 'val_acc'
        )

plt.legend()
plt.show()